# Librerías

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import os
import seaborn as sns

%matplotlib inline

# Datos

In [2]:
train = pd.read_csv('../data/prepro_train.csv')
test = pd.read_csv('../data/prepro_test.csv')

In [3]:
train.head()

,date_number,product_id,block_id,stock,std_stock,family_id,subfamily_id,size_id,color_id,position_max,position_min,std_position,category_id,price,sales
0,0,310130,1726,1383,34.811328,679611953,533441312,7,1,80.0,3.0,37.194534,3,12.95,33
1,0,1178388,592,60,2.160247,732697347,691762817,4,1,19.0,19.0,0.000000,1,49.95,0
2,0,1561460,1625,2373,55.438769,396066037,520569701,5,1,38.0,3.0,14.808299,3,29.95,21
3,0,1874414,1135,1686,20.463906,744793598,811402796,6,1,190.0,12.0,59.873199,6,25.95,24
4,0,2436420,779,245,23.377339,768025921,665805124,5,1,NaN,NaN,NaN,0,25.95,0


In [4]:
test.head()

,date_number,product_id,block_id,stock,std_stock,family_id,subfamily_id,size_id,color_id,position_max,position_min,std_position,category_id,price,sales
0,71,151926,1969,636,33.103206,396066037,335531561,5,1,22.0,5.0,8.959787,2,25.95,-1
1,71,213413,1648,1190,36.225788,552529755,11509337,7,1,139.0,57.0,42.547711,2,19.95,-1
2,71,310130,1726,442,52.809180,679611953,533441312,7,1,46.0,46.0,0.000000,1,12.95,-1
3,71,455200,1400,86,6.831301,998145072,490222156,3,1,53.0,36.0,9.311283,2,29.95,-1
4,71,571044,1098,416,42.178063,831347344,750943270,4,2,205.0,154.0,26.462601,2,15.95,-1


In [5]:
n = 2
d_n = list()
day = list()
for date_n in range(0, 111):
    d_n.append(date_n)
    day.append(n)
    n += 1
    if n >= 7:
        n = 0
        
df_day = pd.DataFrame({'date_number': d_n, 'day': day})
# df_day.head(10)

In [6]:
train = train.join(df_day.set_index('date_number'), on='date_number', how='left')
test = test.join(df_day.set_index('date_number'), on='date_number', how='left')

In [7]:
# n = 0
# m = 2
# d_n = list()
# week = list()
# for date_n in range(0, 111):
#     d_n.append(date_n)
#     week.append(n)
#     m += 1
#     if m >= 7:
#         n += 1
#         m = 0
        
# df_week = pd.DataFrame({'date_number': d_n, 'week': week})
# df_week.head(10)

In [8]:
# train = train.drop('week', axis=1)
# test = test.drop('week', axis=1)

In [9]:
# train = train.join(df_week.set_index('date_number'), on='date_number', how='left')
# test = test.join(df_week.set_index('date_number'), on='date_number', how='left')

In [10]:
# test.head()

In [11]:
# test[test.product_id == 151926]#primera posicion de la primera aparicion

In [12]:
train['weekend'] = np.where((train.day==5) | (train.day==6), 1, 0)
test['weekend'] = np.where((test.day==5) | (test.day==6), 1, 0)
# train.head()

#### Frequency encoding

Para las categorias `family_id` y `subfamily_id`

In [13]:
encoding_train = train.groupby('family_id').size()
encoding_train = encoding_train/len(train)
train['family_id'] = train.family_id.map(encoding_train)
encoding_train = train.groupby('subfamily_id').size()
encoding_train = encoding_train/len(train)
train['subfamily_id'] = train.subfamily_id.map(encoding_train)

encoding_test = test.groupby('family_id').size()
encoding_test = encoding_test/len(test)
test['family_id'] = test.family_id.map(encoding_test)
encoding_test = test.groupby('subfamily_id').size()
encoding_test = encoding_test/len(test)
test['subfamily_id'] = test.subfamily_id.map(encoding_test)

In [14]:
train['stock_lag1'] = train.groupby(['product_id'])['stock'].shift(1)
test['stock_lag1'] = test.groupby(['product_id'])['stock'].shift(1)
train['stock_lead1'] = train.groupby(['product_id'])['stock'].shift(-1)
test['stock_lead1'] = test.groupby(['product_id'])['stock'].shift(-1)

train['std_stock_shift1'] = train[['stock', 'stock_lag1', 'stock_lead1']].std(axis=1)
test['std_stock_shift1'] = test[['stock', 'stock_lag1', 'stock_lead1']].std(axis=1)

train['mean_stock_shift1'] = train[['stock', 'stock_lag1', 'stock_lead1']].mean(axis=1)
test['mean_stock_shift1'] = test[['stock', 'stock_lag1', 'stock_lead1']].mean(axis=1)

train['min_stock_shift1'] = train[['stock', 'stock_lag1', 'stock_lead1']].min(axis=1)
test['min_stock_shift1'] = test[['stock', 'stock_lag1', 'stock_lead1']].min(axis=1)

train['max_stock_shift1'] = train[['stock', 'stock_lag1', 'stock_lead1']].max(axis=1)
test['max_stock_shift1'] = test[['stock', 'stock_lag1', 'stock_lead1']].max(axis=1)

train['median_stock_shift1'] = train[['stock', 'stock_lag1', 'stock_lead1']].median(axis=1)
test['median_stock_shift1'] = test[['stock', 'stock_lag1', 'stock_lead1']].median(axis=1)

train['stock_lag2'] = train.groupby(['product_id'])['stock'].shift(2)
test['stock_lag2'] = test.groupby(['product_id'])['stock'].shift(2)
train['stock_lead2'] = train.groupby(['product_id'])['stock'].shift(-2)
test['stock_lead2'] = test.groupby(['product_id'])['stock'].shift(-2)

train['std_stock_shift2'] = train[['stock', 'stock_lag1', 'stock_lead1',
                                   'stock_lag2', 'stock_lead2']].std(axis=1)
test['std_stock_shift2'] = test[['stock', 'stock_lag1', 'stock_lead1',
                                   'stock_lag2', 'stock_lead2']].std(axis=1)

train['min_stock_shift2'] = train[['stock', 'stock_lag1', 'stock_lead1',
                                   'stock_lag2', 'stock_lead2']].min(axis=1)
test['min_stock_shift2'] = test[['stock', 'stock_lag1', 'stock_lead1',
                                   'stock_lag2', 'stock_lead2']].min(axis=1)

train['max_stock_shift2'] = train[['stock', 'stock_lag1', 'stock_lead1',
                                   'stock_lag2', 'stock_lead2']].max(axis=1)
test['max_stock_shift2'] = test[['stock', 'stock_lag1', 'stock_lead1',
                                   'stock_lag2', 'stock_lead2']].max(axis=1)

train['median_stock_shift2'] = train[['stock', 'stock_lag1', 'stock_lead1',
                                   'stock_lag2', 'stock_lead2']].median(axis=1)
test['median_stock_shift2'] = test[['stock', 'stock_lag1', 'stock_lead1',
                                   'stock_lag2', 'stock_lead2']].median(axis=1)

train['mean_stock_shift2'] = train[['stock', 'stock_lag1', 'stock_lead1',
                                   'stock_lag2', 'stock_lead2']].mean(axis=1)
test['mean_stock_shift2'] = test[['stock', 'stock_lag1', 'stock_lead1',
                                   'stock_lag2', 'stock_lead2']].mean(axis=1)

In [15]:
train['diff_stock_lead1'] = train.stock - train.stock_lead1
train['diff_stock_lead2'] = train.stock - train.stock_lead2
train['diff_stock_lag1'] = train.stock - train.stock_lag1
train['diff_stock_lag2'] = train.stock - train.stock_lag2

test['diff_stock_lead1'] = test.stock - test.stock_lead1
test['diff_stock_lead2'] = test.stock - test.stock_lead2
test['diff_stock_lag1'] = test.stock - test.stock_lag1
test['diff_stock_lag2'] = test.stock - test.stock_lag2

In [16]:
train['std_stock_shift3'] = train[['stock', 'stock_lag1', 'stock_lead1',
                                   'stock_lag2', 'stock_lead2', 'diff_stock_lead1',
                                   'diff_stock_lead2', 'diff_stock_lag1', 'diff_stock_lag2']].std(axis=1)
test['std_stock_shift3'] = test[['stock', 'stock_lag1', 'stock_lead1',
                                   'stock_lag2', 'stock_lead2', 'diff_stock_lead1',
                                   'diff_stock_lead2', 'diff_stock_lag1', 'diff_stock_lag2']].std(axis=1)

train['mean_stock_shift3'] = train[['stock', 'stock_lag1', 'stock_lead1',
                                   'stock_lag2', 'stock_lead2', 'diff_stock_lead1',
                                   'diff_stock_lead2', 'diff_stock_lag1', 'diff_stock_lag2']].mean(axis=1)
test['mean_stock_shift3'] = test[['stock', 'stock_lag1', 'stock_lead1',
                                   'stock_lag2', 'stock_lead2', 'diff_stock_lead1',
                                   'diff_stock_lead2', 'diff_stock_lag1', 'diff_stock_lag2']].mean(axis=1)

train['min_stock_shift3'] = train[['stock', 'stock_lag1', 'stock_lead1',
                                   'stock_lag2', 'stock_lead2', 'diff_stock_lead1',
                                   'diff_stock_lead2', 'diff_stock_lag1', 'diff_stock_lag2']].min(axis=1)
test['min_stock_shift3'] = test[['stock', 'stock_lag1', 'stock_lead1',
                                   'stock_lag2', 'stock_lead2', 'diff_stock_lead1',
                                   'diff_stock_lead2', 'diff_stock_lag1', 'diff_stock_lag2']].min(axis=1)

train['max_stock_shift3'] = train[['stock', 'stock_lag1', 'stock_lead1',
                                   'stock_lag2', 'stock_lead2', 'diff_stock_lead1',
                                   'diff_stock_lead2', 'diff_stock_lag1', 'diff_stock_lag2']].max(axis=1)
test['max_stock_shift3'] = test[['stock', 'stock_lag1', 'stock_lead1',
                                   'stock_lag2', 'stock_lead2', 'diff_stock_lead1',
                                   'diff_stock_lead2', 'diff_stock_lag1', 'diff_stock_lag2']].max(axis=1)

train['median_stock_shift3'] = train[['stock', 'stock_lag1', 'stock_lead1',
                                   'stock_lag2', 'stock_lead2', 'diff_stock_lead1',
                                   'diff_stock_lead2', 'diff_stock_lag1', 'diff_stock_lag2']].median(axis=1)
test['median_stock_shift3'] = test[['stock', 'stock_lag1', 'stock_lead1',
                                   'stock_lag2', 'stock_lead2', 'diff_stock_lead1',
                                   'diff_stock_lead2', 'diff_stock_lag1', 'diff_stock_lag2']].median(axis=1)

In [17]:
train['size_p_color'] = train.size_id * train.color_id
test['size_p_color'] = test.size_id * test.color_id

In [18]:
train['size_d_color'] = train.size_id / train.color_id
test['size_d_color'] = test.size_id / test.color_id

In [19]:
train['diff_position'] = train.position_max - train.position_min
test['diff_position'] = test.position_max - test.position_min

#### Ratios

In [20]:
def gen_ratios(df, c):
    for col in c:
        df['ratio_{}'.format(col)] = train[col] / train[col].max()
        
    return df

In [21]:
cols_ratios = ['median_stock_shift3', 'mean_stock_shift3', 'max_stock_shift3', 'min_stock_shift3',
               'median_stock_shift2', 'mean_stock_shift2', 'max_stock_shift2', 'min_stock_shift2',
               'median_stock_shift1', 'mean_stock_shift1', 'max_stock_shift1', 'min_stock_shift1']

train = gen_ratios(train, cols_ratios)
test = gen_ratios(test, cols_ratios)

In [22]:
train['ratio_std_pos'] = train.std_position / train.std_position.max()
test['ratio_std_pos'] = test.std_position / train.std_position.max()

In [23]:
train['ratio_std_stock'] = train.std_stock / train.std_stock.max()
test['ratio_std_stock'] = test.std_stock / test.std_stock.max()

In [24]:
# train.head()

In [25]:
max_std_train1 = train.groupby('product_id').max()['std_stock_shift1']
max_std_train1.name = 'max_std_stock_shift1'
max_std_test1 = test.groupby('product_id').max()['std_stock_shift1']
max_std_test1.name = 'max_std_stock_shift1'

train = train.join(max_std_train1, on='product_id', how='left')
test = test.join(max_std_test1, on='product_id', how='left')

train['ratio_std_shift1'] = train.std_stock_shift1 / train.max_std_stock_shift1
test['ratio_std_shift1'] = test.std_stock_shift1 / test.max_std_stock_shift1

train = train.drop('max_std_stock_shift1', axis=1)
test = test.drop('max_std_stock_shift1', axis=1)

max_std_train2 = train.groupby('product_id').max()['std_stock_shift2']
max_std_train2.name = 'max_std_stock_shift2'
max_std_test2 = test.groupby('product_id').max()['std_stock_shift2']
max_std_test2.name = 'max_std_stock_shift2'

train = train.join(max_std_train2, on='product_id', how='left')
test = test.join(max_std_test2, on='product_id', how='left')

train['ratio_std_shift2'] = train.std_stock_shift2 / train.max_std_stock_shift2
test['ratio_std_shift2'] = test.std_stock_shift2 / test.max_std_stock_shift2

train = train.drop('max_std_stock_shift2', axis=1)
test = test.drop('max_std_stock_shift2', axis=1)

In [26]:
# train.head()

In [27]:
# train[train.product_id==151926]

#### Frequency encoding por grupos

Útil para series temporales

In [28]:
agrupacion = ['subfamily_id', 'size_id', 'color_id']
a = train.groupby(agrupacion)['product_id'].count()
a.name = 'N1'
train = train.join(a, on=agrupacion, how='left')

a = test.groupby(agrupacion)['product_id'].count()
a.name = 'N1'
test = test.join(a, on=agrupacion, how='left')

In [29]:
agrupacion = ['subfamily_id', 'size_id', 'color_id']
a = train.groupby(agrupacion)['product_id'].count()
a.name = 'N2'
train = train.join(a, on=agrupacion, how='left')

a = test.groupby(agrupacion)['product_id'].count()
a.name = 'N2'
test = test.join(a, on=agrupacion, how='left')

In [30]:
agrupacion = ['subfamily_id', 'family_id']
a = train.groupby(agrupacion)['product_id'].count()
a.name = 'N3'
train = train.join(a, on=agrupacion, how='left')

a = test.groupby(agrupacion)['product_id'].count()
a.name = 'N3'
test = test.join(a, on=agrupacion, how='left')

In [31]:
agrupacion = ['product_id']
a = train.groupby(agrupacion)['date_number'].count()
a.name = 'N5'
train = train.join(a, on=agrupacion, how='left')

a = test.groupby(agrupacion)['date_number'].count()
a.name = 'N5'
test = test.join(a, on=agrupacion, how='left')

In [32]:
agrupacion = ['product_id', 'size_id', 'color_id']
a = train.groupby(agrupacion)['product_id'].count()
a.name = 'N6'
train = train.join(a, on=agrupacion, how='left')

a = test.groupby(agrupacion)['product_id'].count()
a.name = 'N6'
test = test.join(a, on=agrupacion, how='left')

In [33]:
agrupacion = ['size_id', 'color_id']
a = train.groupby(agrupacion)['product_id'].count()
a.name = 'N7'
train = train.join(a, on=agrupacion, how='left')

a = test.groupby(agrupacion)['product_id'].count()
a.name = 'N7'
test = test.join(a, on=agrupacion, how='left')

In [34]:
agrupacion = ['product_id']
a = train.groupby(agrupacion)['family_id'].count()
a.name = 'N8'
train = train.join(a, on=agrupacion, how='left')

a = test.groupby(agrupacion)['family_id'].count()
a.name = 'N8'
test = test.join(a, on=agrupacion, how='left')

In [35]:
agrupacion = ['price', 'family_id', 'subfamily_id']
a = train.groupby(agrupacion)['product_id'].count()
a.name = 'N9'
train = train.join(a, on=agrupacion, how='left')

a = test.groupby(agrupacion)['product_id'].count()
a.name = 'N9'
test = test.join(a, on=agrupacion, how='left')

In [36]:
agrupacion = ['price', 'family_id', 'subfamily_id', 'size_id', 'color_id']
a = train.groupby(agrupacion)['product_id'].count()
a.name = 'N10'
train = train.join(a, on=agrupacion, how='left')

a = test.groupby(agrupacion)['product_id'].count()
a.name = 'N10'
test = test.join(a, on=agrupacion, how='left')

In [37]:
agrupacion = ['price', 'family_id', 'subfamily_id', 'size_id', 'color_id', 'day']
a = train.groupby(agrupacion)['product_id'].count()
a.name = 'N11'
train = train.join(a, on=agrupacion, how='left')

a = test.groupby(agrupacion)['product_id'].count()
a.name = 'N11'
test = test.join(a, on=agrupacion, how='left')

In [38]:
agrupacion = ['block_id', 'size_id', 'color_id']
a = train.groupby(agrupacion)['product_id'].count()
a.name = 'N12'
train = train.join(a, on=agrupacion, how='left')

a = test.groupby(agrupacion)['product_id'].count()
a.name = 'N12'
test = test.join(a, on=agrupacion, how='left')

In [39]:
agrupacion = ['family_id', 'subfamily_id', 'size_id', 'color_id', 'weekend']
a = train.groupby(agrupacion)['product_id'].count()
a.name = 'N13'
train = train.join(a, on=agrupacion, how='left')

a = test.groupby(agrupacion)['product_id'].count()
a.name = 'N13'
test = test.join(a, on=agrupacion, how='left')

In [40]:
# n = 'N12'
# train = train.drop(n, axis=1)
# test = test.drop(n, axis=1)

In [41]:
# train.head()

In [42]:
train.to_csv('../data/final_train.csv', index=None)
test.to_csv('../data/final_test.csv', index=None)

# Prueba de modelo

In [189]:
drop_cols = ['sales', 'date_number', 'product_id', 'block_id',
             'position_max', 'position_min', 'std_position', 'ratio_std_pos', 'diff_position'
            ]

col_sel = [
 'std_stock_shift1',
 'std_stock_shift2',
 'ratio_std_stock',
 'ratio_std_shift1',
 'ratio_std_shift2',]

X = train.loc[:,[c for c in train.columns if c not in drop_cols]]
# X_train = train.loc[:,[c for c in train.columns if c not in drop_cols]]
# X = train[ft_imp]
y = train[['sales']]
# y_train = train[['sales']]

In [190]:
from sklearn.model_selection import train_test_split

In [191]:
# X = X.fillna(-1)

In [192]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [193]:
from sklearn.model_selection import StratifiedKFold
from sklearn.externals import joblib
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error

In [194]:
model_name = 'lgbm'

params = {'max_depth':7,
          'metric':'mae',
          'max_delta_step': 0.2,
          'n_estimators':50000,
          'learning_rate':0.1,
          'colsample_bytree':0.6,
          'objective':'regression',
          'n_jobs':8,
          'seed':42,
          'lambda_l1':0,
          'lambda_l2':0,
#           'max_bin': 14,
#           'bagging_fraction':0.8,
         }

lgb_model = lgb.LGBMRegressor(**params)

In [195]:
train_ids = X_train.index
k = 5
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

In [196]:
counter = 1
be = 0
ft_importances = np.zeros(X_train.shape[1])
full_preds = np.zeros(X_test.shape[0])
for train_index, test_index in skf.split(train_ids, y_train):
    print('--- Fold k {}:'.format(counter))

    X_fit, X_val = X_train.iloc[train_index, :], X_train.iloc[test_index, :]
    y_fit, y_val = y_train.iloc[train_index], y_train.iloc[test_index]
    
    y_val = np.log1p(y_val)
    y_fit = np.log1p(y_fit)

    lgb_model.fit(X_fit,
                  y_fit,
                  eval_set=[(X_val, y_val)],
                  verbose=1000,
                  early_stopping_rounds=50)

    ft_importances += lgb_model.feature_importances_

    be += np.expm1(lgb_model.best_score_['valid_0']['l1'])
#     be += lgb_model.best_score_['valid_0']['l1']
#     print('Score en el test:',mean_absolute_error(lgb_model.predict(X_test), y_test),'ventas')
    y_preds = np.expm1(lgb_model.predict(X_test))
    full_preds += y_preds
    y_preds = [int(round(x)) for x in y_preds]
    print('Score en el test:',round(mean_absolute_error(y_preds, y_test), 3),'"ventas" de error absoluto medio')
    print('Predicciones:', y_preds[:20])
    print('Ventas reales:', y_test[:20].sales.tolist(),'\n\n')
    
    counter += 1
    
# print('\n\nBEST SCORE MEAN:', be / k,'SALES :)')

full_preds = full_preds/k
full_preds = [int(round(x)) for x in full_preds]

print('IMPORTANCIA DE LAS VARIABLES:\n')
ft_importances = lgb_model.feature_importances_
imp = pd.DataFrame({'feature': X_train.columns, 'importance': ft_importances/k})
df_imp_sort = imp.sort_values('importance', ascending=False)
df_imp_sort

--- Fold k 1:
Training until validation scores don't improve for 50 rounds.
[1000]	valid_0's l1: 0.423918
[2000]	valid_0's l1: 0.417649
[3000]	valid_0's l1: 0.415446
Early stopping, best iteration is:
[3091]	valid_0's l1: 0.415236
Score en el test: 5.524 "ventas" de error absoluto medio
Predicciones: [9, 0, 1, 8, 1, 36, 2, 14, 80, 131, 31, 8, 6, 3, 2, 0, 24, 1, 0, 4]
Ventas reales: [5, 0, 0, 7, 4, 27, 4, 22, 117, 176, 30, 4, 16, 7, 2, 0, 9, 1, 0, 4] 


--- Fold k 2:
Training until validation scores don't improve for 50 rounds.
[1000]	valid_0's l1: 0.426138
[2000]	valid_0's l1: 0.419659
[3000]	valid_0's l1: 0.417712
Early stopping, best iteration is:
[3190]	valid_0's l1: 0.417418
Score en el test: 5.565 "ventas" de error absoluto medio
Predicciones: [10, 0, 0, 8, 1, 25, 1, 16, 71, 135, 25, 10, 5, 4, 2, 0, 19, 1, 0, 4]
Ventas reales: [5, 0, 0, 7, 4, 27, 4, 22, 117, 176, 30, 4, 16, 7, 2, 0, 9, 1, 0, 4] 


--- Fold k 3:
Training until validation scores don't improve for 50 rounds.
[1000]	v

,feature,importance
26,diff_stock_lead1,822.6
27,diff_stock_lead2,699.4
25,kurtosis_stock_shift2,670.4
28,diff_stock_lag1,641.2
29,diff_stock_lag2,632.2
39,ratio_std_shift2,564.2
1,std_stock,547.6
38,ratio_std_shift1,530.4
50,N12,507.8
12,std_stock_shift1,499.8


In [ ]:
ft_imp = df_imp_sort.feature.values.tolist()[:10]

# Predicciones por bloque

In [247]:
product_blocks = pd.read_csv('../data/product_blocks.csv')
product_blocks.head(10)

,product_id,block_id
0,612967398,0
1,296892108,0
2,139541214,0
3,963923934,0
4,938230141,0
5,172045154,0
6,663552768,0
7,160621689,1
8,948976891,1
9,556017319,1


In [248]:
productos_por_bloque = product_blocks.groupby('block_id').count()['product_id']
productos_por_bloque.name = 'n_products'
productos_por_bloque.head()

block_id
0     7
1     7
2     7
3     6
4    10
Name: n_products, dtype: int64

In [249]:
product_blocks_n = product_blocks.join(productos_por_bloque, on='block_id', how='left')
product_blocks_n.head()

,product_id,block_id,n_products
0,612967398,0,7
1,296892108,0,7
2,139541214,0,7
3,963923934,0,7
4,938230141,0,7


In [296]:
predicciones = pd.DataFrame({'product_id': train.iloc[X_test.index.values.tolist()].product_id,
                             'preds': y_preds,
                             'price':train.iloc[X_test.index.values.tolist()].price,
                             'gain': y_preds * train.iloc[X_test.index.values.tolist()].price})

predicciones = predicciones.sort_values('gain', ascending=False)
predicciones.head()

,product_id,preds,price,gain
182171,289600941,829,49.95,41408.55
99195,923225371,2241,15.95,35743.95
141229,923225371,1979,15.95,31565.05
196249,400308675,1360,22.95,31212.00
242265,37119458,1336,22.95,30661.20


In [297]:
predicciones_final = predicciones.join(product_blocks_n.set_index('product_id'),
                                       on='product_id', how='left').reset_index(drop=True)
predicciones_final.head()

,product_id,preds,price,gain,block_id,n_products
0,289600941,829,49.95,41408.55,2128,6
1,923225371,2241,15.95,35743.95,2393,9
2,923225371,1979,15.95,31565.05,2393,9
3,400308675,1360,22.95,31212.00,921,5
4,37119458,1336,22.95,30661.20,812,8


In [298]:
group_block_gain = predicciones_final.groupby('block_id').sum().sort_values('gain', ascending=False)['gain']
group_block_gain.name = 'gain_per_block'

In [299]:
preds_final = predicciones_final.join(group_block_gain, on='block_id')
preds_final = preds_final.sort_values('gain_per_block', ascending=False)
preds_final = preds_final.reset_index(drop=True)

In [300]:
preds_final.head()

,product_id,preds,price,gain,block_id,n_products,gain_per_block
0,451648005,21,7.95,166.95,2393,9,214554.35
1,208562180,35,19.95,698.25,2393,9,214554.35
2,208562180,46,19.95,917.70,2393,9,214554.35
3,923225371,1,15.95,15.95,2393,9,214554.35
4,568825435,0,15.95,0.00,2393,9,214554.35


In [301]:
preds_final = preds_final.iloc[preds_final.block_id.drop_duplicates().index.values.tolist()]
preds_final = preds_final.sort_values('gain_per_block', ascending=False)

In [302]:
preds_final.head()

,product_id,preds,price,gain,block_id,n_products,gain_per_block
0,451648005,21,7.95,166.95,2393,9,214554.35
66,679568934,7,12.95,90.65,812,8,174102.10
100,921904870,12,22.95,275.40,2233,8,174059.85
135,112446414,432,25.95,11210.40,2306,3,151885.35
147,846215639,2,12.95,25.90,387,8,138504.80


In [303]:
# predicciones_final = predicciones_final.iloc[predicciones_final.block_id.drop_duplicates().index.values.tolist()]
# predicciones_final.head()

In [305]:
preds_final['product_cumsum'] = preds_final.n_products.cumsum()
preds_final.head()

,product_id,preds,price,gain,block_id,n_products,gain_per_block,product_cumsum
0,451648005,21,7.95,166.95,2393,9,214554.35,9
66,679568934,7,12.95,90.65,812,8,174102.10,17
100,921904870,12,22.95,275.40,2233,8,174059.85,25
135,112446414,432,25.95,11210.40,2306,3,151885.35,28
147,846215639,2,12.95,25.90,387,8,138504.80,36


In [309]:
bet_blocks = preds_final[preds_final.product_cumsum <= 100]
bet_blocks

,product_id,preds,price,gain,block_id,n_products,gain_per_block,product_cumsum
0,451648005,21,7.95,166.95,2393,9,214554.35,9
66,679568934,7,12.95,90.65,812,8,174102.10,17
100,921904870,12,22.95,275.40,2233,8,174059.85,25
135,112446414,432,25.95,11210.40,2306,3,151885.35,28
147,846215639,2,12.95,25.90,387,8,138504.80,36
233,289600941,107,49.95,5344.65,2128,6,138159.70,42
285,310999484,2,9.95,19.90,1874,9,134934.45,51
344,699251225,1,45.95,45.95,1143,10,125950.71,61
408,466243134,333,19.95,6643.35,2000,5,124879.95,66
437,106554518,2,17.95,35.90,388,8,112350.10,74


### Bloques para la apuesta final:

In [307]:
bet_blocks.block_id

0      2393
66      812
100    2233
135    2306
147     387
233    2128
Name: block_id, dtype: int64

In [43]:
from sklearn.linear_model import Lasso

In [44]:
lasso = Lasso()

In [45]:
train_ids = X_train.index
k = 5
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

In [46]:
counter = 1
be = 0
for train_index, test_index in skf.split(train_ids, y_train):
    print('Fold k {}\n'.format(counter))

    X_fit, X_val = X_train.iloc[train_index, :], X_train.iloc[test_index, :]
    y_fit, y_val = y_train.iloc[train_index], y_train.iloc[test_index]
    
    X_fit = X_fit.fillna(-1)
    X_val = X_val.fillna(-1)
    
#     y_val = np.log1p(y_val)
#     y_fit = np.log1p(y_fit)
    
    lasso.fit(X_fit, y_fit)
    
    print('Score en el test:',mean_absolute_error(lasso.predict(X_test.fillna(-1)), y_test),'ventas')
#     print('Score en el test:',mean_absolute_error(np.expm1(lasso.predict(X_test.fillna(-1))), y_test),'ventas')
    
    counter += 1
    
    
# print('\n\nBEST SCORE MEAN:', be / k,'SALES :)')

Fold k 1

Score en el test: 11.170156919880345 ventas
Fold k 2

Score en el test: 11.252111454322224 ventas
Fold k 3



KeyboardInterrupt: 

In [64]:
from sklearn.model_selection import TimeSeriesSplit

In [67]:
tscv = TimeSeriesSplit(n_splits=5)

In [68]:
counter = 1
be = 0
for train_index, test_index in tscv.split(X):
    X_fit, X_val = X.iloc[train_index, :], X.iloc[test_index, :]
    y_fit, y_val = y.iloc[train_index], y.iloc[test_index]
    
#     y_val = np.log1p(y_val)
#     y_fit = np.log1p(y_fit)
    
    
    lgb_model.fit(X_fit,
                  y_fit,
                  eval_set=[(X_val, y_val)],
                  verbose=1000,
                  early_stopping_rounds=20)


#     be += np.expm1(lgb_model.best_score_['valid_0']['l1'])
    be += lgb_model.best_score_['valid_0']['l1']
    
#     print('Score en el test:',mean_absolute_error(lgb_model.predict(X_test), y_test),'ventas')
#     print('Score en el test:',mean_absolute_error(np.expm1(lgb_model.predict(X_test)), y_test),'ventas')
    
    counter += 1

Training until validation scores don't improve for 20 rounds.
[1000]	valid_0's l1: 14.9035
[2000]	valid_0's l1: 14.148
[3000]	valid_0's l1: 13.7446
Early stopping, best iteration is:
[3494]	valid_0's l1: 13.6114
Training until validation scores don't improve for 20 rounds.
[1000]	valid_0's l1: 12.3938
[2000]	valid_0's l1: 11.7195
[3000]	valid_0's l1: 11.3735
Early stopping, best iteration is:
[3420]	valid_0's l1: 11.2791
Training until validation scores don't improve for 20 rounds.
[1000]	valid_0's l1: 10.2273
Early stopping, best iteration is:
[1032]	valid_0's l1: 10.1833
Training until validation scores don't improve for 20 rounds.
Early stopping, best iteration is:
[823]	valid_0's l1: 9.79996
Training until validation scores don't improve for 20 rounds.
[1000]	valid_0's l1: 8.96911
Early stopping, best iteration is:
[1033]	valid_0's l1: 8.93757
